In [7]:
import os
from PIL import Image
import numpy as np
import scipy.io as sio
from torchvision import transforms






In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
# Define paths
raw_images_dir = "ORIGA/ORIGA/raw_images"
annotations_dir = "ORIGA/ORIGA/Semi-automatic-annotations"
output_dir = "ORIGA/ORIGA/processed_images"




In [10]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


In [ ]:
def extract_roi_from_mask(image_path, mat_path, save_path):
    
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)

    
    mat = sio.loadmat(mat_path)
    if 'mask' not in mat:
        print(f"No 'mask' key in {mat_path}")
        return

    mask = mat['mask']

    
    coords = np.argwhere(mask > 0)
    if coords.size == 0:
        print(f"No mask found in {mat_path}")
        return

    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)

    
    roi = img.crop((x_min, y_min, x_max, y_max))
    roi_tensor = transform(roi).to(device) 

    
    roi_img = transforms.ToPILImage()(roi_tensor.cpu())
    roi_img.save(save_path)


In [ ]:
for filename in os.listdir(raw_images_dir):
    if filename.lower().endswith(('.jpg', '.png')):
        base_name = os.path.splitext(filename)[0]
        image_path = os.path.join(raw_images_dir, filename)
        mat_path = os.path.join(annotations_dir, base_name + ".mat")
        save_path = os.path.join(output_dir, base_name + "_roi.jpg")

        if os.path.exists(mat_path):
            try:
                extract_roi_from_mask(image_path, mat_path, save_path)
                print(f"Processed {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")
        else:
            print(f"No annotation found for {filename}")
